In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder,StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
df = pd.read_csv('Clean 2020 Economics.csv')

df.columns = df.columns.str.strip()
df = df.drop(index=0)

df_clean = df[["Income Level", "Reported registered", "Reported not registered"]].copy()
df_clean["Reported registered"] = df_clean["Reported registered"].str.replace(",", "").astype(int)
df_clean["Reported not registered"] = df_clean["Reported not registered"].str.replace(",", "").astype(int)


# Melt the Data 
df_melted = df_clean.melt(id_vars="Income Level", 
                          var_name="Registration Status", 
                          value_name="Count")

# # Step 4: Plot Grouped Bar Chart
# plt.figure(figsize=(12, 6))
# sns.barplot(x="Income Level", y="Count", hue="Registration Status", data=df_melted)

# plt.title("2020 Registration Status by Income Level", fontsize=16)
# plt.xlabel("Income Level", fontsize=12)
# plt.ylabel("Number of People", fontsize=12)
# plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for readability
# plt.legend(title="Registration Status")
# plt.tight_layout()
# plt.show()

df_clean.head()



,Income Level,Reported registered,Reported not registered
1,"Under $10,000",1523,629
2,"$10,000 - $14,999",1745,645
3,"$15,000 - $19,999",1637,787
4,"$20,000 - $29,999",5124,1661
5,"$30,000 - $39,999",7637,1848


In [18]:
# # Step 1: Prepare Data
# def prepare_data(data_path):
#     # Load data
#     df = pd.read_csv(data_path)
#     df.columns = df.columns.str.strip()
    
#     # Select and clean relevant columns
#     df_clean = df[["Income Level", "Reported registered"]].copy()
#     df_clean["Reported registered"] = df_clean["Reported registered"].str.replace(",", "").astype(int)
    
#     # OneHotEncode Income Level
#     encoder = OneHotEncoder(sparse_output=False)  # Corrected argument
#     X_encoded = encoder.fit_transform(df_clean[["Income Level"]])
    
#     # Target Variable
#     y = df_clean["Reported registered"]
    
#     # Normalize features
#     scaler = StandardScaler()
#     X_encoded = scaler.fit_transform(X_encoded)
    
#     # Train-Test Split
#     X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
    
#     # Convert to PyTorch tensors
#     X_train = torch.tensor(X_train, dtype=torch.float32)
#     X_test = torch.tensor(X_test, dtype=torch.float32)
#     y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
#     y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)
    
#     return X_train, X_test, y_train, y_test

# # Step 2: Define Neural Network
# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size):
#         super(NeuralNetwork, self).__init__()
#         self.fc1 = nn.Sequential(
#             nn.Linear(input_size, 64),
#             nn.BatchNorm1d(64),  # Batch normalization
#             nn.ReLU(),
#             nn.Dropout(0.3)      # Dropout for regularization
#         )
#         self.fc2 = nn.Sequential(
#             nn.Linear(64, 32),
#             nn.BatchNorm1d(32),
#             nn.ReLU(),
#             nn.Dropout(0.3)
#         )
#         self.output = nn.Linear(32, 1)  # Output layer
    
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.output(x)
#         return x

# # Step 3: Train the Model
# def train_model(X_train, y_train, X_test, y_test, epochs=50, learning_rate=0.001):
#     model = NeuralNetwork(X_train.shape[1])
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#     criterion = nn.MSELoss()  # Loss function
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # Learning rate scheduler
    
#     for epoch in range(epochs):
#         # Forward pass
#         y_pred = model(X_train)
#         loss = criterion(y_pred, y_train)
        
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         scheduler.step()  # Update learning rate
        
#         if (epoch+1) % 10 == 0:
#             print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
    
#     # Evaluate on test data
#     with torch.no_grad():
#         y_test_pred = model(X_test).detach().numpy()
    
#     mse = mean_squared_error(y_test, y_test_pred)
#     r2 = r2_score(y_test, y_test_pred)
    
#     print("Mean Squared Error (MSE):", mse)
#     print("R2 Score:", r2)
    
#     return model

# # Step 4: Run the Workflow
# data_path = "Clean 2020 Economics.csv"
# X_train, X_test, y_train, y_test = prepare_data(data_path)
# trained_model = train_model(X_train, y_train, X_test, y_test)

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data Preprocessing
# Remove commas and convert relevant columns to numeric
df_clean = df.copy()
for col in ["Total population", "Total citizen population", "Reported registered",
            "Reported not registered", "No response", "Reported voted", 
            "Reported not voted", "No response to voting2"]:
    df_clean[col] = df_clean[col].str.replace(",", "").astype(int)

# One-Hot Encode Income Level
encoder = OneHotEncoder(sparse_output=False, drop="first")  # Avoid multicollinearity by dropping the first category
income_level_encoded = encoder.fit_transform(df_clean[["Income Level"]])

# Create DataFrame for the encoded columns
income_level_encoded_df = pd.DataFrame(
    income_level_encoded, 
    columns=encoder.get_feature_names_out(["Income Level"])
)

# Select features and target
X = pd.concat([
    df_clean[["Total citizen population", "Reported not registered", "Reported voted Percent"]].reset_index(drop=True),
    income_level_encoded_df.reset_index(drop=True)
], axis=1)
y = df_clean["Reported registered"]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Linear Regression Model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2


(21090252.496467076, 0.7490057568509234)